Step 4.1

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import json
import pickle
from datetime import datetime

# Set paths
PROJECT_PATH = '/content/drive/MyDrive/UFC_ML_Project'
DATA_PROCESSED = f'{PROJECT_PATH}/data/processed'
MODELS_PATH = f'{PROJECT_PATH}/models'
VIZ_PATH = f'{PROJECT_PATH}/visualizations'

print("✅ Environment setup complete!")

# Load the best model (XGBoost)
with open(f'{MODELS_PATH}/xgboost_model.pkl', 'rb') as f:
    model = pickle.load(f)

# Load model metadata
with open(f'{MODELS_PATH}/model_metadata.json', 'r') as f:
    metadata = json.load(f)

# Load feature list
feature_list = metadata['feature_list']

# Load historical data for ELO lookups
data = pd.read_csv(f'{DATA_PROCESSED}/elo_ratings.csv')

print(f"✅ Model loaded successfully!")
print(f"   Model: {metadata['best_model']}")
print(f"   Test Accuracy: {metadata['models']['xgboost']['test_accuracy']*100:.2f}%")
print(f"   Features: {len(feature_list)}")

Mounted at /content/drive
✅ Environment setup complete!
✅ Model loaded successfully!
   Model: xgboost
   Test Accuracy: 57.52%
   Features: 39


Step 4.2

In [2]:
def get_fighter_stats(fighter_name, weight_class=None):
    """
    Get the most recent stats for a fighter
    """
    # Filter for this fighter's fights
    fighter_fights = data[(data['R_fighter'] == fighter_name) | (data['B_fighter'] == fighter_name)]

    if len(fighter_fights) == 0:
        return None

    # Get most recent fight
    recent_fight = fighter_fights.iloc[-1]

    # Determine if fighter was Red or Blue
    if recent_fight['R_fighter'] == fighter_name:
        prefix = 'R_'
        elo_before = recent_fight['R_elo_after']  # Use their rating after last fight
        elo_wc_before = recent_fight['R_elo_wc_after']
    else:
        prefix = 'B_'
        elo_before = recent_fight['B_elo_after']
        elo_wc_before = recent_fight['B_elo_wc_after']

    # Extract all stats
    stats = {
        'fighter_name': fighter_name,
        'elo': elo_before,
        'elo_wc': elo_wc_before,
        'last_fight_date': recent_fight['date']
    }

    # Get all average stats
    for col in recent_fight.index:
        if col.startswith(prefix + 'avg_'):
            stat_name = col.replace(prefix, '')
            stats[stat_name] = recent_fight[col]
        elif col.startswith(prefix) and col not in ['R_fighter', 'B_fighter', 'R_Stance', 'B_Stance']:
            stat_name = col.replace(prefix, '')
            stats[stat_name] = recent_fight[col]

    return stats

def calculate_fight_features(red_stats, blue_stats, weight_class, title_bout=False):
    """
    Calculate all features for a fight prediction
    """
    features = {}

    # ELO features
    features['elo_diff'] = red_stats['elo'] - blue_stats['elo']
    features['elo_wc_diff'] = red_stats['elo_wc'] - blue_stats['elo_wc']
    features['elo_avg'] = (red_stats['elo'] + blue_stats['elo']) / 2
    features['elo_wc_avg'] = (red_stats['elo_wc'] + blue_stats['elo_wc']) / 2

    # Physical features
    features['height_diff'] = red_stats['Height_cms'] - blue_stats['Height_cms']
    features['reach_diff'] = red_stats['Reach_cms'] - blue_stats['Reach_cms']
    features['age_diff'] = red_stats['age'] - blue_stats['age']
    features['reach_height_ratio_diff'] = (red_stats['Reach_cms'] / red_stats['Height_cms']) - (blue_stats['Reach_cms'] / blue_stats['Height_cms'])

    # Performance differences
    performance_metrics = ['KD', 'SIG_STR_pct', 'TD_pct', 'SUB_ATT', 'CTRL_time(seconds)',
                          'SIG_STR_landed', 'TD_landed']

    for metric in performance_metrics:
        key = f'avg_{metric}'
        if key in red_stats and key in blue_stats:
            features[f'diff_{metric}'] = red_stats[key] - blue_stats[key]

    # Experience features
    features['diff_total_fights'] = (red_stats['wins'] + red_stats['losses']) - (blue_stats['wins'] + blue_stats['losses'])
    features['diff_wins'] = red_stats['wins'] - blue_stats['wins']
    features['diff_losses'] = red_stats['losses'] - blue_stats['losses']

    # Win rates
    red_win_rate = red_stats['wins'] / (red_stats['wins'] + red_stats['losses'] + 0.1)
    blue_win_rate = blue_stats['wins'] / (blue_stats['wins'] + blue_stats['losses'] + 0.1)
    features['diff_win_rate'] = red_win_rate - blue_win_rate

    # Streaks
    features['diff_win_streak'] = red_stats['current_win_streak'] - blue_stats['current_win_streak']
    features['diff_lose_streak'] = red_stats['current_lose_streak'] - blue_stats['current_lose_streak']

    # Title bouts
    features['title_bout'] = 1 if title_bout else 0
    features['diff_title_bouts'] = red_stats['total_title_bouts'] - blue_stats['total_title_bouts']

    # Weight class encoding
    weight_classes = ['Bantamweight', 'CatchWeight', 'Featherweight', 'Flyweight',
                     'Heavyweight', 'LightHeavyweight', 'Lightweight', 'Middleweight',
                     'OpenWeight', 'Welterweight', 'WomenBantamweight', 'WomenFeatherweight',
                     'WomenFlyweight', 'WomenStrawweight']

    for wc in weight_classes:
        features[f'weight_class_{wc}'] = 1 if weight_class == wc else 0

    # Stance encoding (simplified - you may need to adjust based on actual stances)
    features['orthodox_vs_southpaw'] = 0  # Would need actual stance data

    # Fill any missing features with 0
    for feat in feature_list:
        if feat not in features:
            features[feat] = 0

    return features

# Test the functions
print("🧪 Testing fighter lookup...")
test_fighter = data['R_fighter'].iloc[-1]
test_stats = get_fighter_stats(test_fighter)
if test_stats:
    print(f"✅ Successfully retrieved stats for {test_fighter}")
    print(f"   ELO: {test_stats['elo']:.0f}")
    print(f"   Record: {test_stats['wins']:.0f}-{test_stats['losses']:.0f}")
else:
    print(f"❌ Fighter not found")

🧪 Testing fighter lookup...
✅ Successfully retrieved stats for Adrian Yanez
   ELO: 1556
   Record: 1-0


Step 4.3

In [3]:
def predict_fight(red_fighter, blue_fighter, weight_class, title_bout=False):
    """
    Predict the outcome of a UFC fight

    Args:
        red_fighter: Name of the red corner fighter
        blue_fighter: Name of the blue corner fighter
        weight_class: Weight class of the fight
        title_bout: Boolean indicating if it's a title fight

    Returns:
        Dictionary with prediction results
    """

    # Get fighter stats
    red_stats = get_fighter_stats(red_fighter, weight_class)
    blue_stats = get_fighter_stats(blue_fighter, weight_class)

    # Check if both fighters exist
    if red_stats is None:
        return {
            'error': f"Fighter '{red_fighter}' not found in database",
            'success': False
        }

    if blue_stats is None:
        return {
            'error': f"Fighter '{blue_fighter}' not found in database",
            'success': False
        }

    # Calculate features
    features = calculate_fight_features(red_stats, blue_stats, weight_class, title_bout)

    # Ensure features are in the correct order
    feature_vector = [features.get(feat, 0) for feat in feature_list]
    feature_df = pd.DataFrame([feature_vector], columns=feature_list)

    # Make prediction
    prediction = model.predict(feature_df)[0]
    probabilities = model.predict_proba(feature_df)[0]

    # Calculate confidence
    confidence = abs(probabilities[1] - 0.5) * 2 * 100  # Convert to percentage

    # Determine winner
    if prediction == 1:
        predicted_winner = red_fighter
        win_probability = probabilities[1] * 100
    else:
        predicted_winner = blue_fighter
        win_probability = probabilities[0] * 100

    # Prepare result
    result = {
        'success': True,
        'prediction': {
            'red_fighter': red_fighter,
            'blue_fighter': blue_fighter,
            'weight_class': weight_class,
            'title_bout': title_bout,
            'predicted_winner': predicted_winner,
            'red_win_probability': round(probabilities[1] * 100, 2),
            'blue_win_probability': round(probabilities[0] * 100, 2),
            'confidence': round(confidence, 2),
            'prediction_timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        },
        'fighter_stats': {
            'red_fighter': {
                'elo': round(red_stats['elo'], 0),
                'elo_weight_class': round(red_stats['elo_wc'], 0),
                'record': f"{int(red_stats['wins'])}-{int(red_stats['losses'])}",
                'win_streak': int(red_stats['current_win_streak']),
                'height_cm': round(red_stats['Height_cms'], 1),
                'reach_cm': round(red_stats['Reach_cms'], 1)
            },
            'blue_fighter': {
                'elo': round(blue_stats['elo'], 0),
                'elo_weight_class': round(blue_stats['elo_wc'], 0),
                'record': f"{int(blue_stats['wins'])}-{int(blue_stats['losses'])}",
                'win_streak': int(blue_stats['current_win_streak']),
                'height_cm': round(blue_stats['Height_cms'], 1),
                'reach_cm': round(blue_stats['Reach_cms'], 1)
            }
        },
        'key_factors': {
            'elo_advantage': f"{predicted_winner} (+{abs(round(features['elo_diff'], 0))} ELO)",
            'physical_advantage': f"Reach: {red_fighter if features['reach_diff'] > 0 else blue_fighter} (+{abs(round(features['reach_diff'], 1))} cm)",
            'experience_advantage': f"{red_fighter if features['diff_total_fights'] > 0 else blue_fighter} (+{abs(int(features['diff_total_fights']))} fights)"
        }
    }

    return result

# Test prediction function
print("\n🧪 Testing prediction function...")
print("="*80)

# Get two recent fighters
test_red = data['R_fighter'].iloc[-5]
test_blue = data['B_fighter'].iloc[-5]
test_weight_class = data['weight_class'].iloc[-5]

prediction_result = predict_fight(test_red, test_blue, test_weight_class)

if prediction_result['success']:
    print(f"✅ Prediction successful!")
    print(f"\n🥊 {test_red} vs {test_blue}")
    print(f"   Weight Class: {test_weight_class}")
    print(f"\n🏆 Predicted Winner: {prediction_result['prediction']['predicted_winner']}")
    print(f"   Confidence: {prediction_result['prediction']['confidence']:.1f}%")
    print(f"   {test_red} Win Probability: {prediction_result['prediction']['red_win_probability']:.1f}%")
    print(f"   {test_blue} Win Probability: {prediction_result['prediction']['blue_win_probability']:.1f}%")
else:
    print(f"❌ Prediction failed: {prediction_result['error']}")


🧪 Testing prediction function...
✅ Prediction successful!

🥊 Song Kenan vs Max Griffin
   Weight Class: Welterweight

🏆 Predicted Winner: Song Kenan
   Confidence: 1.0%
   Song Kenan Win Probability: 50.5%
   Max Griffin Win Probability: 49.5%


Step 4.4

In [4]:
def predict_multiple_fights(fights_list):
    """
    Predict outcomes for multiple fights

    Args:
        fights_list: List of dictionaries, each containing:
            - red_fighter
            - blue_fighter
            - weight_class
            - title_bout (optional)

    Returns:
        List of prediction results
    """
    results = []

    for fight in fights_list:
        result = predict_fight(
            fight['red_fighter'],
            fight['blue_fighter'],
            fight['weight_class'],
            fight.get('title_bout', False)
        )
        results.append(result)

    return results

# Example usage
print("\n🧪 Testing batch predictions...")
print("="*80)

test_fights = [
    {
        'red_fighter': data['R_fighter'].iloc[-1],
        'blue_fighter': data['B_fighter'].iloc[-1],
        'weight_class': data['weight_class'].iloc[-1]
    },
    {
        'red_fighter': data['R_fighter'].iloc[-2],
        'blue_fighter': data['B_fighter'].iloc[-2],
        'weight_class': data['weight_class'].iloc[-2]
    }
]

batch_results = predict_multiple_fights(test_fights)

for i, result in enumerate(batch_results, 1):
    if result['success']:
        print(f"\n{i}. {result['prediction']['red_fighter']} vs {result['prediction']['blue_fighter']}")
        print(f"   Winner: {result['prediction']['predicted_winner']} ({result['prediction']['confidence']:.1f}% confidence)")


🧪 Testing batch predictions...

1. Adrian Yanez vs Gustavo Lopez
   Winner: Adrian Yanez (12.3% confidence)

2. Leonardo Santos vs Grant Dawson
   Winner: Leonardo Santos (5.9% confidence)


Step 4.5

In [5]:
def save_prediction(prediction_result):
    """Save prediction to history file"""

    predictions_file = f'{PROJECT_PATH}/data/predictions/prediction_history.csv'

    # Flatten prediction result for CSV
    flat_result = {
        'timestamp': prediction_result['prediction']['prediction_timestamp'],
        'red_fighter': prediction_result['prediction']['red_fighter'],
        'blue_fighter': prediction_result['prediction']['blue_fighter'],
        'weight_class': prediction_result['prediction']['weight_class'],
        'title_bout': prediction_result['prediction']['title_bout'],
        'predicted_winner': prediction_result['prediction']['predicted_winner'],
        'red_win_probability': prediction_result['prediction']['red_win_probability'],
        'blue_win_probability': prediction_result['prediction']['blue_win_probability'],
        'confidence': prediction_result['prediction']['confidence'],
        'red_elo': prediction_result['fighter_stats']['red_fighter']['elo'],
        'blue_elo': prediction_result['fighter_stats']['blue_fighter']['elo']
    }

    # Check if file exists
    try:
        existing_predictions = pd.read_csv(predictions_file)
        predictions_df = pd.concat([existing_predictions, pd.DataFrame([flat_result])], ignore_index=True)
    except FileNotFoundError:
        predictions_df = pd.DataFrame([flat_result])

    # Save
    predictions_df.to_csv(predictions_file, index=False)
    print(f"✅ Prediction saved to {predictions_file}")

# Test save function
if prediction_result['success']:
    save_prediction(prediction_result)

✅ Prediction saved to /content/drive/MyDrive/UFC_ML_Project/data/predictions/prediction_history.csv


Step 4.7

In [6]:
!pip install flask pyngrok -q

from flask import Flask, request, jsonify
from pyngrok import ngrok
import threading
import time
import pandas as pd
import pickle
import json
import numpy as np

ngrok.set_auth_token("37rIInNv5kHeBNDQJe8USUMDvcB_55k95jjmkst1DqrNYaSr")

try:
    ngrok.kill()
    time.sleep(1)
except:
    pass

print("📦 Loading model and data...")

PROJECT_PATH = '/content/drive/MyDrive/UFC_ML_Project'
MODELS_PATH = f'{PROJECT_PATH}/models'
DATA_PROCESSED = f'{PROJECT_PATH}/data/processed'

with open(f'{MODELS_PATH}/xgboost_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

with open(f'{MODELS_PATH}/model_metadata.json', 'r') as f:
    loaded_metadata = json.load(f)

loaded_data = pd.read_csv(f'{DATA_PROCESSED}/elo_ratings.csv')
loaded_data['date'] = pd.to_datetime(loaded_data['date'])
loaded_features = loaded_metadata['feature_list']

elo_ratings = {}
for idx, row in loaded_data.iterrows():
    elo_ratings[row['R_fighter']] = row['R_elo_after']
    elo_ratings[row['B_fighter']] = row['B_elo_after']

print(f"✅ Model: {loaded_metadata['best_model']}, Fighters: {len(elo_ratings)}\n")

def get_fighter_latest_stats(fighter_name):
    fighter_fights = loaded_data[(loaded_data['R_fighter'] == fighter_name) |
                                  (loaded_data['B_fighter'] == fighter_name)]
    if len(fighter_fights) == 0:
        return None

    recent = fighter_fights.iloc[-1]
    prefix = 'R_' if recent['R_fighter'] == fighter_name else 'B_'

    stats = {
        'fighter_name': fighter_name,
        'elo': float(recent[f'{prefix}elo_after']),
        'elo_wc': float(recent[f'{prefix}elo_wc_after']),
        'Height_cms': float(recent[f'{prefix}Height_cms']),
        'Reach_cms': float(recent[f'{prefix}Reach_cms']),
        'age': float(recent[f'{prefix}age']),
        'wins': float(recent[f'{prefix}wins']),
        'losses': float(recent[f'{prefix}losses']),
        'current_win_streak': float(recent[f'{prefix}current_win_streak']),
        'current_lose_streak': float(recent[f'{prefix}current_lose_streak']),
        'total_title_bouts': float(recent[f'{prefix}total_title_bouts'])
    }

    for col in recent.index:
        if col.startswith(f'{prefix}avg_'):
            stat_name = col.replace(f'{prefix}', '')
            stats[stat_name] = float(recent[col])

    return stats

def calculate_features_for_prediction(red_stats, blue_stats, weight_class, title_bout):
    features = {}

    features['elo_diff'] = red_stats['elo'] - blue_stats['elo']
    features['elo_wc_diff'] = red_stats['elo_wc'] - blue_stats['elo_wc']
    features['elo_avg'] = (red_stats['elo'] + blue_stats['elo']) / 2
    features['elo_wc_avg'] = (red_stats['elo_wc'] + blue_stats['elo_wc']) / 2

    features['height_diff'] = red_stats['Height_cms'] - blue_stats['Height_cms']
    features['reach_diff'] = red_stats['Reach_cms'] - blue_stats['Reach_cms']
    features['age_diff'] = red_stats['age'] - blue_stats['age']
    features['reach_height_ratio_diff'] = (red_stats['Reach_cms']/red_stats['Height_cms']) - (blue_stats['Reach_cms']/blue_stats['Height_cms'])

    perf_metrics = ['KD', 'SIG_STR_pct', 'TD_pct', 'SUB_ATT', 'CTRL_time(seconds)']
    for m in perf_metrics:
        key = f'avg_{m}'
        if key in red_stats and key in blue_stats:
            features[f'diff_{m}'] = red_stats[key] - blue_stats[key]

    features['diff_wins'] = red_stats['wins'] - blue_stats['wins']
    features['diff_losses'] = red_stats['losses'] - blue_stats['losses']
    features['diff_win_streak'] = red_stats['current_win_streak'] - blue_stats['current_win_streak']
    features['title_bout'] = 1 if title_bout else 0

    weight_classes = ['Bantamweight', 'Featherweight', 'Flyweight', 'Heavyweight',
                     'LightHeavyweight', 'Lightweight', 'Middleweight', 'Welterweight']
    for wc in weight_classes:
        features[f'weight_class_{wc}'] = 1 if weight_class == wc else 0

    for feat in loaded_features:
        if feat not in features:
            features[feat] = 0

    return features

def make_prediction(red_fighter, blue_fighter, weight_class, title_bout=False):
    red_stats = get_fighter_latest_stats(red_fighter)
    blue_stats = get_fighter_latest_stats(blue_fighter)

    if not red_stats:
        return {'success': False, 'error': f"Fighter '{red_fighter}' not found"}
    if not blue_stats:
        return {'success': False, 'error': f"Fighter '{blue_fighter}' not found"}

    features = calculate_features_for_prediction(red_stats, blue_stats, weight_class, title_bout)
    X = pd.DataFrame([[features[f] for f in loaded_features]], columns=loaded_features)

    prediction = int(loaded_model.predict(X)[0])
    probabilities = loaded_model.predict_proba(X)[0]

    result = {
        'success': True,
        'prediction': {
            'red_fighter': red_fighter,
            'blue_fighter': blue_fighter,
            'weight_class': weight_class,
            'title_bout': bool(title_bout),
            'predicted_winner': red_fighter if prediction == 1 else blue_fighter,
            'red_win_probability': round(float(probabilities[1]) * 100, 2),
            'blue_win_probability': round(float(probabilities[0]) * 100, 2),
            'confidence': round(float(abs(probabilities[1] - 0.5)) * 2 * 100, 2),
            'prediction_timestamp': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')
        },
        'fighter_stats': {
            'red_fighter': {
                'elo': int(round(red_stats['elo'])),
                'record': f"{int(red_stats['wins'])}-{int(red_stats['losses'])}",
                'height_cm': round(red_stats['Height_cms'], 1),
                'reach_cm': round(red_stats['Reach_cms'], 1),
                'win_streak': int(red_stats['current_win_streak'])
            },
            'blue_fighter': {
                'elo': int(round(blue_stats['elo'])),
                'record': f"{int(blue_stats['wins'])}-{int(blue_stats['losses'])}",
                'height_cm': round(blue_stats['Height_cms'], 1),
                'reach_cm': round(blue_stats['Reach_cms'], 1),
                'win_streak': int(blue_stats['current_win_streak'])
            }
        }
    }

    return result

app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def api_predict():
    try:
        data = request.get_json()
        if not all(k in data for k in ['red_fighter', 'blue_fighter', 'weight_class']):
            return jsonify({'success': False, 'error': 'Missing required fields'}), 400

        result = make_prediction(data['red_fighter'], data['blue_fighter'],
                                data['weight_class'], data.get('title_bout', False))
        return jsonify(result)
    except Exception as e:
        return jsonify({'success': False, 'error': str(e)}), 500

@app.route('/batch-predict', methods=['POST'])
def api_batch_predict():
    try:
        data = request.get_json()
        fights = data.get('fights', [])
        results = [make_prediction(f['red_fighter'], f['blue_fighter'],
                                   f['weight_class'], f.get('title_bout', False))
                  for f in fights]
        return jsonify({'success': True, 'predictions': results, 'total': len(results)})
    except Exception as e:
        return jsonify({'success': False, 'error': str(e)}), 500

@app.route('/health', methods=['GET'])
def health():
    return jsonify({
        'status': 'healthy',
        'model': loaded_metadata['best_model'],
        'test_accuracy': f"{loaded_metadata['models']['xgboost']['test_accuracy']*100:.2f}%",
        'total_fighters': len(elo_ratings)
    })

@app.route('/', methods=['GET'])
def index():
    return jsonify({'message': 'UFC Fight Prediction API',
                   'endpoints': ['GET /', 'GET /health', 'POST /predict', 'POST /batch-predict']})

def run_flask():
    app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)

print("⏳ Starting server...")
flask_thread = threading.Thread(target=run_flask, daemon=True)
flask_thread.start()
time.sleep(3)

tunnel = ngrok.connect(5000, bind_tls=True)
NGROK_URL = tunnel.public_url

print(f"\n{'='*80}\n🎉 API LIVE!\n{'='*80}")
print(f"URL: {NGROK_URL}\nFor n8n: {NGROK_URL}/predict\n{'='*80}\n")

📦 Loading model and data...
✅ Model: xgboost, Fighters: 1676

⏳ Starting server...
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit



🎉 API LIVE!
URL: https://monocultural-nondictatorially-gage.ngrok-free.dev
For n8n: https://monocultural-nondictatorially-gage.ngrok-free.dev/predict



Step 4.8

In [7]:
import requests
import json

print("🧪 Testing API\n")

# Test 1: Health
response = requests.get(f"{NGROK_URL}/health")
print(f"1️⃣ Health: {response.status_code}")
if response.status_code == 200:
    print(f"   ✅ {response.json()}")

# Test 2: Prediction
payload = {
    "red_fighter": "Jon Jones",
    "blue_fighter": "Stipe Miocic",
    "weight_class": "Heavyweight"
}

response = requests.post(f"{NGROK_URL}/predict", json=payload)
print(f"\n2️⃣ Prediction: {response.status_code}")
if response.status_code == 200:
    result = response.json()
    if result['success']:
        print(f"   ✅ Winner: {result['prediction']['predicted_winner']}")
        print(f"   Confidence: {result['prediction']['confidence']}%")
    else:
        print(f"   ❌ {result['error']}")

print(f"\n🔗 n8n URL: {NGROK_URL}/predict")

🧪 Testing API



INFO:werkzeug:127.0.0.1 - - [08/Jan/2026 20:52:29] "GET /health HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Jan/2026 20:52:29] "POST /predict HTTP/1.1" 200 -


1️⃣ Health: 200
   ✅ {'model': 'xgboost', 'status': 'healthy', 'test_accuracy': '57.52%', 'total_fighters': 1676}

2️⃣ Prediction: 200
   ✅ Winner: Jon Jones
   Confidence: 14.1%

🔗 n8n URL: https://monocultural-nondictatorially-gage.ngrok-free.dev/predict


Step 4.9

In [8]:
# ============================================================================
# FINAL API TEST
# ============================================================================

import requests
import json

print("🧪 Testing your UFC Prediction API\n")
print("="*80)

# Test 1: Health Check
print("\n1️⃣ Health Check...")
try:
    response = requests.get(f"{NGROK_URL}/health", timeout=10)
    print(f"   Status Code: {response.status_code}")
    if response.status_code == 200:
        result = response.json()
        print(f"   ✅ API is healthy!")
        print(f"   Model: {result['model']}")
        print(f"   Accuracy: {result['test_accuracy']}")
        print(f"   Fighters: {result['total_fighters']}")
    else:
        print(f"   ❌ Failed: {response.text}")
except Exception as e:
    print(f"   ❌ Error: {e}")

# Test 2: Make a Real Prediction
print("\n2️⃣ Making a prediction...")
try:
    # Get two recent fighters from your data
    test_fight = data.iloc[-10]  # Use a fight from 10 rows back

    payload = {
        "red_fighter": test_fight['R_fighter'],
        "blue_fighter": test_fight['B_fighter'],
        "weight_class": test_fight['weight_class'],
        "title_bout": bool(test_fight['title_bout'])
    }

    print(f"   Fight: {payload['red_fighter']} vs {payload['blue_fighter']}")
    print(f"   Weight Class: {payload['weight_class']}")

    response = requests.post(
        f"{NGROK_URL}/predict",
        headers={"Content-Type": "application/json"},
        json=payload,
        timeout=30
    )

    print(f"   Status Code: {response.status_code}")

    if response.status_code == 200:
        result = response.json()
        if result.get('success'):
            print(f"   ✅ Prediction successful!")
            print(f"\n   🥊 PREDICTION RESULTS:")
            print(f"   Predicted Winner: {result['prediction']['predicted_winner']}")
            print(f"   Confidence: {result['prediction']['confidence']}%")
            print(f"   {payload['red_fighter']}: {result['prediction']['red_win_probability']}%")
            print(f"   {payload['blue_fighter']}: {result['prediction']['blue_win_probability']}%")

            print(f"\n   📊 Fighter Stats:")
            print(f"   {payload['red_fighter']} - ELO: {result['fighter_stats']['red_fighter']['elo']}, Record: {result['fighter_stats']['red_fighter']['record']}")
            print(f"   {payload['blue_fighter']} - ELO: {result['fighter_stats']['blue_fighter']['elo']}, Record: {result['fighter_stats']['blue_fighter']['record']}")
        else:
            print(f"   ❌ Prediction failed: {result.get('error')}")
    else:
        print(f"   ❌ HTTP Error: {response.text[:300]}")

except Exception as e:
    print(f"   ❌ Error: {e}")

# Test 3: Test with a custom matchup
print("\n3️⃣ Testing custom matchup...")
try:
    # Find two popular fighters
    fighter_list = data['R_fighter'].value_counts().head(20).index.tolist()

    custom_payload = {
        "red_fighter": fighter_list[0],
        "blue_fighter": fighter_list[1],
        "weight_class": "Lightweight"
    }

    print(f"   Custom fight: {custom_payload['red_fighter']} vs {custom_payload['blue_fighter']}")

    response = requests.post(
        f"{NGROK_URL}/predict",
        json=custom_payload,
        timeout=30
    )

    if response.status_code == 200 and response.json().get('success'):
        result = response.json()
        print(f"   ✅ Predicted winner: {result['prediction']['predicted_winner']}")
        print(f"   Confidence: {result['prediction']['confidence']}%")
    else:
        print(f"   ⚠️ Response: {response.json()}")

except Exception as e:
    print(f"   ❌ Error: {e}")

print("\n" + "="*80)
print("🎉 API TESTING COMPLETE!")
print("="*80)
print(f"\n🔗 Your API URL for n8n:")
print(f"   {NGROK_URL}/predict")
print("\n📋 Next step: Set up n8n workflow")
print("="*80)

INFO:werkzeug:127.0.0.1 - - [08/Jan/2026 20:52:29] "GET /health HTTP/1.1" 200 -


🧪 Testing your UFC Prediction API


1️⃣ Health Check...
   Status Code: 200
   ✅ API is healthy!
   Model: xgboost
   Accuracy: 57.52%
   Fighters: 1676

2️⃣ Making a prediction...
   Fight: Trevin Giles vs Roman Dolidze
   Weight Class: Middleweight


INFO:werkzeug:127.0.0.1 - - [08/Jan/2026 20:52:30] "POST /predict HTTP/1.1" 200 -


   Status Code: 200
   ✅ Prediction successful!

   🥊 PREDICTION RESULTS:
   Predicted Winner: Trevin Giles
   Confidence: 0.89%
   Trevin Giles: 50.45%
   Roman Dolidze: 49.55%

   📊 Fighter Stats:
   Trevin Giles - ELO: 1547, Record: 4-2
   Roman Dolidze - ELO: 1517, Record: 2-0

3️⃣ Testing custom matchup...
   Custom fight: Donald Cerrone vs Jim Miller


INFO:werkzeug:127.0.0.1 - - [08/Jan/2026 20:52:30] "POST /predict HTTP/1.1" 200 -


   ✅ Predicted winner: Donald Cerrone
   Confidence: 4.74%

🎉 API TESTING COMPLETE!

🔗 Your API URL for n8n:
   https://monocultural-nondictatorially-gage.ngrok-free.dev/predict

📋 Next step: Set up n8n workflow
